In [1]:
import napari_workflows
from napari_workflows._io_yaml_v1 import load_workflow
import napari
import napari_pyclesperanto_assistant
from napari_pyclesperanto_assistant._workflow_io_utility import *
from skimage.io import imread

filename = 'test_workflow_multiple_nodes_no_sum.yaml'
workflow = load_workflow(filename)
print(workflow)

Workflow:
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x0000019563507B80>, 'blobs', 5.0)
Result of Gaussian (scikit-image, nsbatwm) [1] <- (<function gaussian_blur at 0x0000019563507B80>, 'blobs_messed_up', 13.0)
Result of Seeded watershed using local minima as seeds (nsbatwm) <- (<function local_minima_seeded_watershed at 0x000001956350BE50>, 'Result of Gaussian (scikit-image, nsbatwm)', 2.0, 2.0)
Result of White top-hat (scipy, nsbatwm) <- (<function white_tophat at 0x000001956350B040>, 'Result of Gaussian (scikit-image, nsbatwm) [1]', 21.0)



In [2]:
#desktop = 'C:/Users/ryans/Desktop/'
desktop = 'C:/Users/ryans/Onedrive/Desktop/'

image1 = imread(desktop + 'blobs.tif')
image2 = imread(desktop + 'blobs_messed_up.tiff')

viewer = napari.Viewer()
viewer.add_image(image1)
viewer.add_image(image2)

C:\Users\ryans\Anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'image2' at 0x19563609790>

In [3]:
initialise_root_functions(workflow=workflow, 
                          viewer=viewer)

In [4]:
load_remaining_workflow(workflow=workflow,viewer=viewer)

TypeError: unsupported operand type(s) for &: 'float' and 'int'

TypeError: unsupported operand type(s) for &: 'float' and 'int'

TypeError: unsupported operand type(s) for &: 'float' and 'int'

TypeError: unsupported operand type(s) for &: 'float' and 'int'

TypeError: unsupported operand type(s) for &: 'float' and 'int'